<a href="https://colab.research.google.com/github/amilkh/cs230-fer/blob/datagen/fer2013_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#tensorflow_version 1.x

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.lib.io import file_io

import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

%matplotlib inline

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


try:
    pydot.Dot.create(pydot.Dot())
except:
    print('pydot error')

In [4]:
print(tf.__version__)
print(keras.__version__)

1.15.0
2.2.5


data = pd.read_csv('fer2013/fer2013.csv')
data.head()

data_train = data[data['Usage'] == 'Training']
#print('Number samples in the training dataset: ', data_train.shape[0])

data_dev = data[data['Usage'] == 'PublicTest']
#print('Number samples in the development dataset: ', data_dev.shape[0])
data_dev.head()

data_train.to_csv('fer2013/train.csv')
data_dev.to_csv('fer2013/dev.csv')

print(data_train.shape)

print(data_dev.shape)

In [0]:
EPOCHS = 300
BS = 128
DROPOUT_RATE = 0.4
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

data = pd.read_csv('fer2013/icml_face_data.csv') 
print(data.head())
print(data.shape)

print(data[' Usage'].unique())

data_dev2 = data[data[' Usage'] == 'PublicTest']
#print('Number samples in the development dataset: ', data_dev.shape[0])
print(data_dev2.head())

data_dev2.to_csv('fer2013/dev2.csv')
print(data_dev2.shape)

data_test = data[data[' Usage'] == 'PrivateTest']
#print('Number samples in the development dataset: ', data_dev.shape[0])
print(data_test.head())

data_test.to_csv('fer2013/test.csv')
print(data_test.shape)

Resize_pixelsize = 197
vgg_notop = VGGFace(model='resnet50', include_top=False, input_shape=(Resize_pixelsize, Resize_pixelsize, 3), pooling='avg')
last_layer = vgg_notop.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(4096, activation='relu', name='fc6')(x)
x = Dense(1024, activation='relu', name='fc7')(x)
#print("Emotions count", len(EMOTIONS))
l=0
for layer in vgg_notop.layers:
    print(layer,"["+str(l)+"]")
    l=l+1
    
batch_norm_indices = [2, 6, 9, 13, 14, 18, 21, 24, 28, 31, 34, 38, 41, 45, 46, 53, 56, 60, 63, 66, 70, 73, 76, 80, 83, 87, 88, 92, 95, 98]
for i in range(101):
    if i not in batch_norm_indices:
        vgg_notop.layers[i].trainable = False

print('vgg layer 2 is trainable: ' + str(vgg_notop.layers[2].trainable))
print('vgg layer 3 is trainable: ' + str(vgg_notop.layers[3].trainable))

out = Dense(7, activation='softmax', name='classifier')(x)

custom_resnet = Model(vgg_notop.input, out)


optim = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optim = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

custom_resnet.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
plot_model(custom_resnet, to_file='model2.png', show_shapes=True)

In [6]:
# Implement below paper CPCPCPFF depth 5, 2.4m params
# http://openaccess.thecvf.com/content_cvpr_2016_workshops/w28/papers/Kim_Fusing_Aligned_and_CVPR_2016_paper.pdf
# Reference: https://arxiv.org/pdf/1612.02903.pdf
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu',padding='same', input_shape=(48,48,1),name="conv1"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool1"))
#model.add(Dropout(DROPOUT_RATE))
model.add(Conv2D(32, (4, 4), activation='relu',padding='same',name="conv2"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))         
#model.add(Dropout(DROPOUT_RATE))
model.add(Conv2D(64, (5, 5), activation='relu',padding='same',name="conv3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model.add(Flatten())
model.add(Dense(1024, activation='relu',name='fc1'))
model.add(Dropout(DROPOUT_RATE))
model.add(Dense(7, activation='softmax',name='fcsoftmax'))

#TODO: weight decay of 0.0001...initial learning rate is set to 0.01 and reduced by a factor of 2 at every 25 epoch
sgd = SGD(lr=SGD_LEARNING_RATE,momentum=0.9, decay=SGD_DECAY, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
#rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100)
rlrop = ReduceLROnPlateau(monitor='val_acc',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
Batch_no = 0
# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
    # dataset: Data path
def get_data(datas, bs= 32, aug=None):
    global Batch_no
    # Data preparation
    while True:
      file_stream = file_io.FileIO(training_dataset_dir, mode='r')
      datas = pd.read_csv(file_stream,iterator=True, chunksize=bs )
      for data in datas:
          #data = pd.read_csv('fer2013/fer2013.csv')
          data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])

          # Retrieve train input and target
          X, Y = data['pixels'].tolist(), data['emotion'].values
          #print(len(X))
          #print(X[0])
          # Reshape images to 4D (num_samples, width, height, num_channels)
          X = np.array(X, dtype='float32').reshape(-1,48,48,1)
          # Normalize images with max (the maximum pixel intensity is 255)
          X_res = X/255.0
          #print(X.shape)
          #print(X[0])
          #image_resized = resize(image, (image.shape[0] // 4, image.shape[1] // 4), anti_aliasing=True)

          Y_res = np.zeros((Y.size, 7))
          Y_res[np.arange(Y.size),Y] = 1
          # if the data augmentation object is not None, apply it
          if aug is not None:
              (X_res, Y_res) = next(aug.flow(np.array(X_res),
                  Y_res, batch_size=bs))
          # if the data augmentation object is not None, apply it
          print(Batch_no)
          Batch_no += 1
          yield  X_res, Y_res

In [0]:
EPOCHS = 20
BS = 128

training_dataset_dir = '/content/drive/My Drive/cs230 project/collab/fer2013/train.csv'
dev_dataset_dir = '/content/drive/My Drive/cs230 project/collab/fer2013/dev.csv'

aug = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)



train_generator = get_data(training_dataset_dir,  bs=BS, aug=aug)
dev_generator   = get_data(dev_dataset_dir, bs=BS, aug=None)
    #X_dev_res, Y_dev_res  = get_data(dev_dataset_dir)

# Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches) indefinitely
# rescale:          Rescaling factor (defaults to None). Multiply the data by the value provided (before applying any other transformation)
# rotation_range:   Int. Degree range for random rotations
# shear_range:      Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
# zoom_range:       Float or [lower, upper]. Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range]
# fill_mode :       Points outside the boundaries of the input are filled according to the given mode: {"constant", "nearest", "reflect" or "wrap"}
# horizontal_flip:  Boolean. Randomly flip inputs horizontally


# Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
    # x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
    #               it should have value 3
    # y:            Labels
    # batch_size:   Int (default: 32)
#train_generator = train_datagen.flow(X_train_res, Y_train_res,  batch_size  = BS)

small_x = X_train_res[0:5000]
small_y = Y_train_res[0:5000]

In [13]:
history = model.fit_generator(
    generator = train_generator,
    validation_data=dev_generator, 
    steps_per_epoch=28709// BS,
    shuffle=True,
    epochs=EPOCHS  ) 

ValueError: ignored

custom_resnet.fit_generator(
    generator           = train_generator,
    validation_data=dev_generator, 
    steps_per_epoch=(28709) // BS,
    shuffle=True,
    epochs=EPOCHS) 

In [0]:
H = custom_resnet.fit_generator(
    train_generator,
    steps_per_epoch=(28709) // BS,
    validation_data=dev_generator,
    validation_steps=3589 // BS,
    epochs=EPOCHS)

In [0]:
custom_resnet.save('transfer_learning_DG.h5')

custom_resnet.evaluate(X_train_res, Y_train_res)

custom_resnet.evaluate(X_dev_res, Y_dev_res)